In [5]:
!pip install python-telegram-bot
!pip install torch transformers
!pip install python-telegram-bot --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 37.3 MB/s eta 0:00:00


In [42]:
from transformers import pipeline
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import asyncio
import nest_asyncio

nest_asyncio.apply()

text_generator = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_new_tokens=300,
    device="cuda",
    do_sample=True,
    temperature=0.7,
    pad_token_id=50256,
    eos_token_id=50256,
    truncation=True
)

def truncate_to_full_stop(text: str, max_chars: int) -> str:
    """
    Truncate the text to the last full stop before reaching the max character limit.
    """
    if len(text) <= max_chars:
        return text
    truncated = text[:max_chars]
    last_full_stop = truncated.rfind(".")
    if last_full_stop != -1:
        return truncated[:last_full_stop + 1]
    return truncated

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        "Hi there! I’m your AI assistant powered by TinyLlama. 😊\n"
        "Ask me anything or share your thoughts, and I'll do my best to help!"
    )

async def process(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    processing_message = await update.message.reply_text("Let me think...")

    try:
        user_input = f"<human>: {update.message.text}\n<assistant>:"
        generated = text_generator(user_input, num_return_sequences=1)
        full_response = generated[0]["generated_text"]

        response_text = full_response.split("<assistant>:")[-1].strip()
        if "<human>:" in response_text:
            response_text = response_text.split("<human>:")[0].strip()

        response_text = truncate_to_full_stop(response_text, max_chars=300)

        if len(response_text) < 10:
            response_text = "Hmm, could you rephrase that? I didn’t quite get it."

        await processing_message.delete()
        await update.message.reply_text(response_text)

    except Exception as e:
        error_message = f"Oops, something went wrong: {str(e)}"
        await processing_message.delete()
        await update.message.reply_text(error_message)

async def main():
    TOKEN = "7626859522:AAEoBP0ysfamXjG1-7UqpvXhVBpKDYE0Uf4"
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, process))

    print("Bot is running... Press Ctrl+C to stop.")
    await application.run_polling()

loop = asyncio.get_event_loop()
if not loop.is_running():
    loop.run_until_complete(main())
else:
    asyncio.ensure_future(main())


Device set to use cuda
ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 750, in _network_loop_retry
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 744, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 371, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line

Bot is running... Press Ctrl+C to stop.
